<a href="https://colab.research.google.com/github/tummalapallimurali/GenAI/blob/main/BERT_Keyword_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Find Similar sentences from the text.
- Extract keywords to find similarity using BERT

In [3]:
!pip install -U sentence-transformers

In [4]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


In [5]:
text = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised.
Deep learning architectures such as deep neural networks, deep belief networks, recurrent neural networks and convolutional neural networks have been applied to fields including computer vision, machine vision, speech recognition, natural language processing, audio recognition, social network filtering, machine translation, bioinformatics, drug design, medical image analysis, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains. Specifically, neural networks tend to be static and symbolic, while the biological brain of most living organisms is dynamic (plastic) and analog.
The adjective "deep" in deep learning comes from the use of multiple layers in the network. Early work showed that a linear perceptron cannot be a universal classifier, and then that a network with a nonpolynomial activation function with one hidden layer of unbounded width can on the other hand so be. Deep learning is a modern variation which is concerned with an unbounded number of layers of bounded size, which permits practical application and optimized implementation, while retaining theoretical universality under mild conditions. In deep learning the layers are also permitted to be heterogeneous and to deviate widely from biologically informed connectionist models, for the sake of efficiency, trainability and understandability, whence the "structured" part.
"""

print(text)

Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised.
Deep learning architectures such as deep neural networks, deep belief networks, recurrent neural networks and convolutional neural networks have been applied to fields including computer vision, machine vision, speech recognition, natural language processing, audio recognition, social network filtering, machine translation, bioinformatics, drug design, medical image analysis, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
Artificial neural networks (ANNs) were inspired by information processing and distributed communication nodes in biological systems. ANNs have various differences from biological brains. Specifically, neural networks tend to be static a

- Set nGramRange to set size of resulting keyphrase candidates or KeyWords that we require/


In [7]:
nGramRange = (1, 1) # set range of candidates we require.
stop_words = "english" # only english
# using sklearn countvectorizer etract keywords/phrases.
count = CountVectorizer(ngram_range=nGramRange, stop_words=stop_words).fit([text])

print(count)

CountVectorizer(stop_words='english')


In [9]:
canidates = count.get_feature_names_out()
print(canidates)

['activation' 'adjective' 'analog' 'analysis' 'anns' 'application'
 'applied' 'architectures' 'artificial' 'audio' 'based' 'belief'
 'bioinformatics' 'biological' 'biologically' 'board' 'bounded' 'brain'
 'brains' 'broader' 'cases' 'classifier' 'comes' 'communication'
 'comparable' 'computer' 'concerned' 'conditions' 'connectionist'
 'convolutional' 'deep' 'design' 'deviate' 'differences' 'distributed'
 'drug' 'dynamic' 'early' 'efficiency' 'expert' 'family' 'fields'
 'filtering' 'function' 'game' 'hand' 'heterogeneous' 'hidden' 'human'
 'image' 'implementation' 'including' 'information' 'informed'
 'inspection' 'inspired' 'known' 'language' 'layer' 'layers' 'learning'
 'linear' 'living' 'machine' 'material' 'medical' 'methods' 'mild'
 'models' 'modern' 'multiple' 'natural' 'network' 'networks' 'neural'
 'nodes' 'nonpolynomial' 'number' 'optimized' 'organisms' 'perceptron'
 'performance' 'permits' 'permitted' 'plastic' 'practical' 'processing'
 'produced' 'programs' 'recognition' 'recu

✅ Some of models available for \
Semantic Textual Similarity:
1. roberta-large-nli-stsb-mean-tokens
2. roberta-base-nli-stsb-mean-tokens
3. bert-large-nli-stsb-mean-tokens
4. distilbert-base-nli-stsb-mean-tokens

Semantic Similarity
1. distiluse-base-multilingual-cased-v2
2. xlm-r-distilroberta-base-paraphrase-v1
3. xlm-r-bert-base-nli-stsb-mean-tokens
4. distilbert-multilingual-nli-stsb-quora-ranking

# Text classification using BERT

BERT requires input specific format:
1. inputids, segment mask, attention mask, labels

1. BERT can be used for a widevariety of language tasks, while adding one layer to core model
2. For Sentiment analysis and **next sentence classification, add classifiaction layer** on top of the transformer output.
3. In Q & A - Question regarding a text sequence and is requried to mark the answer in sequence.
4. NER --> Text sequence,(Person, Organizaition, date). NER model can be trained by feeding output vector of each token into a classification layer.

In [1]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.8 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split



from tqdm import tqdm
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt


In [3]:
# load bert tokenizer

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Data Preprocessing

In [4]:
df = pd.read_csv('/content/in_domain_train.tsv',
                 sep='\t',
                 header=None,
                 names = ['sentence_source', 'label', 'label_notes', 'sentence'])

print(df.shape)

df.head()

# convert sentences to list, we need to pass CLS and SEP so each sentence is embedded based on start and end

sentences = df.sentence.values

sentences = ["[CLS]" + sentence + "[SEP]" for sentence in sentences]
labels = df.label.values

# Loop each sentence and tokenize each sentence in the list

tokenized_text = [tokenizer.tokenize(sent) for sent in sentences]

print(tokenized_text[0])


(8551, 4)
['[CLS]', 'our', 'friends', 'won', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.', '[SEP]']


In [5]:
df

,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"Our friends won't buy this analysis, let alone..."
1,gj04,1,NaN,One more pseudo generalization and I'm giving up.
2,gj04,1,NaN,One more pseudo generalization or I'm giving up.
3,gj04,1,NaN,"The more we study verbs, the crazier they get."
4,gj04,1,NaN,Day by day the facts are getting murkier.
...,...,...,...,...
8546,ad03,0,*,Poseidon appears to own a dragon
8547,ad03,0,*,Digitize is my happiest memory
8548,ad03,1,NaN,It is easy to slay the Gorgon.
8549,ad03,1,NaN,I had the strangest feeling that I knew you.


1. BERT requires format inputs in following format:
  - input ids(index no in the bert tokenizer library)
  - segment mask(0's and 1's to identify if its one sentence or 2 sentences. if one sentence this is simply 0's. If 2 , for 1st sentence it will be 0's and 2nd sentence it will be 1's)
  - attention mask(a seq of 1;s and 0's, 1's of all input tokens, 0's for padding tokens)
  - labels( 1 or 0)

2. Padding: Input array should be same size. , they are set to max_len if sentence is < max length, then rest of it is set to 0's.
3. Pad_seq is used from keras.



In [6]:
# Set Max Len
MAX_LEN = 128

# input ids

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]
print(input_ids[2])

# pad input tokens

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(input_ids[2])

# create attention masks

attention_masks = []


for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

print(attention_masks[2])

# train and test split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels,
                                                                                    random_state=2018,
                                                                                    test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks,
                                                       input_ids,
                                                       random_state=2018,
                                                       test_size=0.1)
# Convert train_masks to a tensor
train_masks = torch.tensor(train_masks)
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)

#convert valiations to tensor
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)
validation_inputs = torch.tensor(validation_inputs)


# Convert to TensorDataset
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

# Similar change for validation_masks


'''convert validation dataset to tensordataset and dataloader is to iterate enitre dataset without
loading into memory, dataloader will bundle inputids, attention mask and labels in batches.
and one tensor converts to triples of input ids, attention mask and labels.

(tensor([ 101, 2002, 2939, 1996, 3328, 1012,  102,0 , 0 ....]),
 tensor([1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0]),
 tensor(1))
'''

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=32)

print(train_data[0])

'''

1. Modify pretrainedd BERT to give outputs for classification.
2. training end-to-end suited for our task.
3. Each Toplayers are output types are designed for specific NLP task.
4. BERTForSequenceClassification: Normal bert model with an added single layer ontop for classification for sentence classifier.

Structure of fine tuning model:
1. [the hidden state , number of labels] are passed to outoput of bert plus classification layer
2. vector representing the "score" for grammatical/non grammatical is returned.
3. This is fed to cross entropy loss.

FineTuning process:
1. 95% of bottowm layers are already trained,
2. we need train top layer, with tweeking
3. some times "FREEZE" option is used to freeze bottom layers.
4. or Apply differene learning rates for good quality weights.
5.

'''

from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# hyperparameters






[101, 2028, 2062, 18404, 2236, 3989, 2030, 1045, 1005, 1049, 3228, 2039, 1012, 102]
[  101  2028  2062 18404  2236  3989  2030  1045  1005  1049  3228  2039
  1012   102     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# get parameter values for weight_decay_rate

# get parameter values for weight_decay_rate

[name for name, _ in model.named_parameters() if name in ['bias']] # Call named_parameters() to get the iterable

[]

In [8]:
for name, param in model.named_parameters():
  if name in ['bias']:
    print(param.size())



In [9]:
# Define hyperparameters
learning_rate = 2e-5  # Example learning rate
weight_decay = 0.01   # Example weight decay
epsilon = 1e-8        # Example epsilon value

# set hypertuning parameters
# Create optimizer with hyperparameters
optimizer = AdamW(model.parameters(),
                     lr=learning_rate,
                     weight_decay=weight_decay,
                     eps=epsilon)





/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



For each pass in the training loop we have a training phase and a validation phase.

At each pass we need to:

Training loop:

Tell the model to compute gradients by setting the model in train mode
Unpack our data inputs and labels
Load data onto the GPU for acceleration
Clear out the gradients calculated in the previous pass. In pytorch the gradients accumulate by default (useful for things like RNNs) unless you explicitly clear them out
Forward pass (feed input data through the network)
Backward pass (backpropagation)
Tell the network to update parameters with optimizer.step()
Track variables for monitoring progress
Evalution loop:

Tell the model not to compute gradients by setting th emodel in evaluation mode
Unpack our data inputs and labels
Load data onto the GPU for acceleration
Forward pass (feed input data through the network)
Compute loss on our validation data and track variables for monitoring progress

In [ ]:
# Start Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loss  = []
epochs= 2

for _ in range(epochs):
  total_loss = 0
  model.train()
  tr_loss = 0
  nb_tr_steps = 0
  nb_tr_examples = 0

  for step, batch in enumerate(train_dataloader):
    #batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    optimizer.zero_grad()
    outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels)
    loss = outputs[0]
    total_loss += loss.item()
    loss.backward()
    optimizer.step()
    tr_loss += loss.item()
    nb_tr_steps += 1
    nb_tr_examples += b_input_ids.size(0)
    if step % 100 == 0:
      loss_step = tr_loss/nb_tr_steps
      print(f"Training loss after {step}: {loss_step}")

  print(f"Training loss epoch: {epoch}: {tr_loss/nb_tr_steps}")
  loss.append(tr_loss/nb_tr_steps)

  # validation
  model.eval()
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
      outputs = model(b_input_ids,
                      token_type_ids=None,
                      attention_mask=b_input_mask)
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      tmp_eval_accuracy = flat_accuracy(logits, label_ids)
      eval_accuracy += tmp_eval_accuracy
      nb_eval_steps += 1
  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))




Training loss after 0: 0.7783048748970032
